In [ ]:
!pip install pyspark google-api-python-client kafka-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 4.6 MB/s eta 0:00:00


In [ ]:
pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00


In [ ]:
pip install regex

In [ ]:
pip install emot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.5 MB/s eta 0:00:00


In [ ]:
pip install openpyxl


In [ ]:
import numpy as np
import pandas as pd
from pyvi.ViTokenizer import ViTokenizer
import regex as re
import os
import matplotlib.pyplot as plt
import seaborn as sns
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
from collections import Counter
import random

In [ ]:
STOPWORDS = 'drive/MyDrive/IE212_project/content/vietnamese-stopwords-dash.txt'
with open(STOPWORDS, "r", encoding="utf-8") as ins:
    stopwords = []
    for line in ins:
        dd = line.strip('\n')
        stopwords.append(dd)
    stopwords = set(stopwords)

def filter_stop_words(train_sentences, stop_words):
    new_sent = [word for word in train_sentences.split() if word not in stop_words]
    train_sentences = ' '.join(new_sent)
    return train_sentences

def check_repeated_character(text):
    text = re.sub('  +', ' ', text).strip()
    count = {}
    for i in range(len(text) - 1):
        if text[i] == text[i + 1]:
            return True
    return False

def check_space(text):  # check space in string
    for i in range(len(text)):
        if text[i] == ' ':
            return True
    return False


def check_special_character_numberic(text):
    return any(not c.isalpha() for c in text)

def remove_emoji(text):
    for emot in UNICODE_EMOJI:
        text = str(text).replace(emot, ' ')
    text = re.sub('  +', ' ', text).strip()
    return text

# Remove url
def url(text):
    text = re.sub(r'https?://\S+|www\.\S+', ' ', str(text))
    text = re.sub('  +', ' ', text).strip()
    return text

# remove special character
def special_character(text):
    text = re.sub(r'\d+', lambda m: " ", text)
    # text = re.sub(r'\b(\w+)\s+\1\b',' ', text) #remove duplicate number word
    text = re.sub("[~!@#$%^&*()_+{}“”|:\"<>?`´\-=[\]\;\\\/.,]", " ", text)
    text = re.sub('  +', ' ', text).strip()
    return text

# normalize repeated characters
def repeated_character(text):
    text = re.sub(r'(\w)\1+', r'\1', text)
    text = re.sub('  +', ' ', text).strip()
    return text

def mail(text):
    text = re.sub(r'[^@]+@[^@]+\.[^@]+', ' ', text)
    text = re.sub('  +', ' ', text).strip()
    return text

# remove mention tag and hashtag
def tag(text):
    text = re.sub(r"(?:\@|\#|\://)\S+", " ", text)
    text = re.sub('  +', ' ', text).strip()
    return text

# """Remove all mixed words and numbers"""
def mixed_word_number(text):
    text = ' '.join(s for s in text.split() if not any(c.isdigit() for c in s))
    text = re.sub('  +', ' ', text).strip()
    return text

c2e_path = os.path.join(os.getcwd(), 'drive/MyDrive/IE212_project/content/dictionary/character2emoji.xlsx')
character2emoji = pd.read_excel(c2e_path)  # character to emoji
def convert_character2emoji(text):
    text = str(text)
    for i in range(character2emoji.shape[0]):
        text = text.replace(character2emoji.at[i, 'character'], " " + character2emoji.at[i, 'emoji'] + " ")
    text = re.sub('  +', ' ', text).strip()
    return text

e2w_path = os.path.join(os.getcwd(), 'drive/MyDrive/IE212_project/content/dictionary/emoji2word.xlsx')
emoji2word = pd.read_excel(e2w_path)  # emoji to word

def convert_emoji2word(text):
    for i in range(emoji2word.shape[0]):
        text = text.replace(emoji2word.at[i, 'emoji'], " " + emoji2word.at[i, 'word_vn'] + " ")
    text = re.sub('  +', ' ', text).strip()
    return text

adn_path = os.path.join(os.getcwd(), 'drive/MyDrive/IE212_project/content/dictionary/abb_dict_normal.xlsx')
abb_dict_normal = pd.read_excel(adn_path)


def abbreviation_normal(text):  # len word equal 1
    text = str(text)
    temp = ''
    for word in text.split():
        for i in range(abb_dict_normal.shape[0]):
            if str(abb_dict_normal.at[i, 'abbreviation']) == str(word):
                word = str(abb_dict_normal.at[i, 'meaning'])
        temp = temp + ' ' + word
    text = temp
    text = re.sub('  +', ' ', text).strip()
    return text

ads_path = os.path.join(os.getcwd(), 'drive/MyDrive/IE212_project/content/dictionary/abb_dict_special.xlsx')
abb_dict_special = pd.read_excel(ads_path)


def abbreviation_special(text):  # including special character and number
    text = ' ' + str(text) + ' '
    for i in range(abb_dict_special.shape[0]):
        text = text.replace(' ' + abb_dict_special.at[i, 'abbreviation'] + ' ',
                            ' ' + abb_dict_special.at[i, 'meaning'] + ' ')
    text = re.sub('  +', ' ', text).strip()
    return text

def special_character_1(text):  # remove dot and comma
    text = re.sub("[.,?!]", " ", text)
    text = re.sub('  +', ' ', text).strip()
    return text

def abbreviation_kk(text):
    text = str(text)
    for t in text.split():
        if 'kk' in t:
            text = text.replace(t, ' ha ha ')
        else:
            if 'kaka' in t:
                text = text.replace(t, ' ha ha ')
            else:
                if 'kiki' in t:
                    text = text.replace(t, ' ha ha ')
                else:
                    if 'haha' in t:
                        text = text.replace(t, ' ha ha ')
                    else:
                        if 'hihi' in t:
                            text = text.replace(t, ' ha ha ')
    text = re.sub('  +', ' ', text).strip()
    return text

# tokenize by lib Pyvi
def tokenize(text):
    text = str(text)
    text = ViTokenizer.tokenize(text)
    return text


In [ ]:
def preprocessing(text, lowercased = True):
    text = text.lower() if lowercased else text
    text = convert_character2emoji(text)
    text = url(text)
    text = mail(text)
    text = tag(text)
    text = mixed_word_number(text)
    text = special_character_1(text)  # ##remove , . ? !
    text = abbreviation_kk(text)
    text = abbreviation_special(text)
    text = convert_character2emoji(text)
    text = remove_emoji(text)
    text = repeated_character(text)
    text = special_character(text)
    text = abbreviation_normal(text)
    text = tokenize(text)
    text = filter_stop_words(text, stopwords)
    return text

In [ ]:
df = pd.read_csv('drive/MyDrive/IE212_project/content/Real_Time_Comment/Youtube_comments.csv')

In [ ]:

# Áp dụng hàm preprocessing cho cột "comment"
df['comment_preprocessing'] = df['Comment'].apply(lambda x: preprocessing(x))
df.drop(columns=['Comment'], inplace=True)
df.dropna()
# Lưu kết quả vào file CSV mới
df.to_csv('drive/MyDrive/IE212_project/content/Youtube_comment_after_preprocessing/Youtube_comments_after_preprocessing.csv', index=False)

In [ ]:
data_preprocessing = pd.read_csv('drive/MyDrive/IE212_project/content/Youtube_comment_after_preprocessing/Youtube_comments_after_preprocessing.csv')

In [ ]:
data_preprocessing.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1930 entries, 0 to 1929
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   comment_preprocessing  1894 non-null   object 
 1   comment_length         1894 non-null   float64
dtypes: float64(1), object(1)
memory usage: 30.3+ KB


In [ ]:
# Assuming the column you want to examine is called 'comment_preprocessing'
data_preprocessing['comment_length'] = data_preprocessing['comment_preprocessing'].str.len()

# Print the DataFrame with the new 'comment_length' column
print(data_preprocessing[['comment_preprocessing', 'comment_length']])

                                  comment_preprocessing  comment_length
0     ơi đãi năm mới gói trọn_vẹn giảm chỉ từ mua tạ...           156.0
1                 runo là tiền_vệ ngu nhất mọi thời_đại            37.0
2                 thích mu nhưng ghét thằng ăn_hại runo            37.0
3                                       tôi phan mu năm            15.0
4                                           mu_mai dinh            11.0
...                                                 ...             ...
1925  chúc_mừng mu có điểm đầy quý_giá và rất vất_vả...            84.0
1926                                                NaN             NaN
1927                                                 up             2.0
1928                                           đầu_tiên             8.0
1929                                                NaN             NaN

[1930 rows x 2 columns]


In [ ]:
import pyspark
from IPython.display import display, clear_output
from pyspark.sql import functions as f
from pyspark.sql.functions import udf
from pyspark.sql.streaming import DataStreamReader
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
# Định dạng cho timestamp
timestampformat = 'EEE MMM dd HH:mm:ss zzzz yyyy'

# Tạo một SparkSession với tên ứng dụng
spark = SparkSession.builder.appName("StructuredStreamingExample").getOrCreate()

# Thiết lập chính sách phân tích thời gian Legacy
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")



# Thiết lập lại chính sách thời gian
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")




In [ ]:
INPATH = "drive/MyDrive/IE212_project/content/Youtube_comment_after_preprocessing/"

In [ ]:
# Define schema explicitly
schema = StructType([
    StructField("comment_preprocessing", StringType(), True) # Assuming your column is of StringType
])

In [ ]:
spark_reader = spark.readStream.schema(schema)

In [ ]:
streaming_data = (
    spark_reader.csv(INPATH)
    .select("comment_preprocessing")
    .coalesce(1)
)

In [ ]:
stream_writer = (streaming_data
                 .writeStream
                 .queryName("data")
                 .trigger(once=True)
                 .option("header", "false")
                 .outputMode("append")
                 .format("memory"))

In [ ]:
query = stream_writer.start()

In [ ]:
display(spark.sql(f"SELECT * from {query.name}").show(truncate=False))

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|comment_preprocessing                                                                                                                                            |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|comment_preprocessing                                                                                                                                            |
|ơi đãi năm mới gói trọn_vẹn giảm chỉ từ mua tại 𝐒𝐡𝐨𝐩𝐞 nhận voucher giảm cho gói tháng và cơ_hội trúng quà may_mắn với tổng giá_trị tới triệu đồng đừng bỏ lỡ|
|runo là tiền_vệ ngu nhất mọi thời_đại                                                                                                                            |
|thích mu nhưng ghét 

None

In [ ]:
from pyspark.ml import PipelineModel
dt_model = PipelineModel.load("drive/MyDrive/IE212_project/content/DecisionTree")

In [ ]:
from pyspark.sql.functions import split, col, array_join, when, size, array, lit

# ... (your existing code) ...

# Bước 1: Tách cột 'words' thành mảng các từ và xử lý null/empty
streaming_data = streaming_data.withColumn("words",
                                          when(col("comment_preprocessing") != "", split(col("comment_preprocessing"), " "))
                                          .otherwise(array(lit(""))) # Replace null/empty with an empty array
                                         )

# Bước 2: Áp dụng mô hình RandomForest để dự đoán
predictions = dt_model.transform(streaming_data)

# Bước 3: Chọn các cột cần thiết và lọc ra các hàng có words rỗng
result_df = predictions.select(
    array_join(col("words"), " ").alias("joined_words"),  # Gộp mảng thành chuỗi và đặt tên mới là 'joined_words'
    "prediction"
).filter(size(col("words")) > 0) # Filter out rows with empty words using the original 'words' column

# ... (rest of your code for writing the stream) ...

In [ ]:
stream_writer = (result_df
                 .writeStream
                 .queryName("data")
                 .trigger(once=True)
                 .outputMode("append")
                 .format("memory"))

In [ ]:
query = stream_writer.start()

In [ ]:
display(spark.sql(f"SELECT * from {query.name}").show())

+--------------------+----------+
|        joined_words|prediction|
+--------------------+----------+
|comment_preproces...|       0.0|
|ơi đãi năm mới gó...|       1.0|
|runo là tiền_vệ n...|       0.0|
|thích mu nhưng gh...|       0.0|
|     tôi phan mu năm|       0.0|
|         mu_mai dinh|       0.0|
|chỉ cần một_chút ...|       0.0|
|thủ_môn pha cuối ...|       0.0|
|cần thanh_lọc đội...|       0.0|
|mẹ mu càng ngày_c...|       0.0|
|khuyến_mãi to vậy...|       0.0|
|       tết đến có lo|       0.0|
|mot cau thu tuyet...|       0.0|
|        fan man city|       0.0|
|          tôi fan mu|       0.0|
|làm_ơn hãy tạc tư...|       0.0|
|hàng công phun ph...|       0.0|
|hãy đên với để đư...|       0.0|
|    đúng kèo nhà_cái|       0.0|
|từ ngày biết đến ...|       0.0|
+--------------------+----------+
only showing top 20 rows



None

In [ ]:
prediction_counts_clean = (
    result_df.filter(
        (col("prediction") == 0.0)
    )
    .agg(f.count("prediction").alias("count_clean"))
)

In [ ]:
prediction_counts_hate = (
    result_df.filter(
        (col("prediction") == 1.0)
    )
    .agg(f.count("prediction").alias("count_harmful"))
)

In [ ]:
data_to_stream_hate = prediction_counts_hate
data_to_stream_clean = prediction_counts_clean

In [ ]:
stream_writer_clean = (data_to_stream_clean
                 .writeStream
                 .queryName("streaming_table_new_0")
                 .trigger(processingTime="20 seconds")
                 .outputMode("complete")
                 .format("memory"))

In [ ]:
stream_writer_hate = (data_to_stream_hate
                 .writeStream
                 .queryName("streaming_table_new_1")
                 .trigger(processingTime="20 seconds")
                 .outputMode("complete")
                 .format("memory"))

In [ ]:
if query.isActive:
    query.stop()
query = stream_writer_clean.start()


In [ ]:
display(spark.sql(f"SELECT * FROM {query.name}").show())

+-----------+
|count_clean|
+-----------+
|       1821|
+-----------+



None

In [ ]:
if query.isActive:
    query.stop()
query = stream_writer_hate.start()

In [ ]:
display(spark.sql(f"SELECT * FROM {query.name}").show())

+-------------+
|count_harmful|
+-------------+
|          110|
+-------------+



None

In [ ]:
filtered_df_hate = (
    result_df.filter(col("prediction") == 1.0)
             .select("joined_words")
)
filtered_df_clean = (
    result_df.filter(col("prediction") == 0.0)
             .select("joined_words")
)

In [ ]:
stream_writer_hate = (filtered_df_hate
                 .writeStream
                 .queryName("data_hate")
                 .trigger(once=True)
                 .outputMode("append")
                 .format("memory"))
stream_writer_clean = (filtered_df_clean
                 .writeStream
                 .queryName("data_clean")
                 .trigger(once=True)
                 .outputMode("append")
                 .format("memory"))

In [ ]:
query_hate = stream_writer_hate.start()
query_clean = stream_writer_clean.start()

In [ ]:
display(spark.sql(f"SELECT * FROM {query_hate.name}").show(truncate=False))


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|joined_words                                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ơi đãi năm mới gói trọn_vẹn giảm chỉ từ mua tại 𝐒𝐡𝐨𝐩𝐞 nhận voucher giảm cho gói tháng và cơ_hội trúng quà may_mắn với tổng giá_trị tới triệu đồng đừng bỏ lỡ   |
|mu_ngoài amad casemiro thì mấy thằng còn lại rất điệu và tích màu_mè                                                                                                |
|ông kiến_tạo quả thứ cho amad là con rể roy kean                                                                                                                    |
|vã

None

In [ ]:
display(spark.sql(f"SELECT * FROM {query_clean.name}").show(truncate=False))

+----------------------------------------------------------------------------------------------+
|joined_words                                                                                  |
+----------------------------------------------------------------------------------------------+
|comment_preprocessing                                                                         |
|runo là tiền_vệ ngu nhất mọi thời_đại                                                         |
|thích mu nhưng ghét thằng ăn_hại runo                                                         |
|tôi phan mu năm                                                                               |
|mu_mai dinh                                                                                   |
|chỉ cần một_chút thời_gian giúp tôi quên hết âu_lo và tận_hưởng những khoảnh_khắc vui_vẻ      |
|thủ_môn pha cuối ăn_mừng                                                                      |
|cần thanh_lọc đội_hình       

None

In [ ]:
# Write the streaming data to memory
stream_writer = (result_df
                 .writeStream
                 .queryName("result_df_table")
                 .trigger(once=True)  # Trigger only once to process all available data
                 .outputMode("append")
                 .format("memory"))

query = stream_writer.start()
query.awaitTermination()  # Wait for the stream to finish processing

# Read the data back as a static DataFrame
static_df = spark.sql("SELECT * FROM result_df_table")

# Now you can use .collect() or toPandas() on the static DataFrame
collected_df = static_df.collect()

static_df.write.csv("drive/MyDrive/IE212_project/content/predictions_spark_streaming", header=True, mode="overwrite")

In [ ]:
import pandas as pd
import os

# Đường dẫn tới thư mục chứa file CSV
directory_path = 'drive/MyDrive/IE212_project/content/predictions_spark_streaming'

# Lưu danh sách các DataFrame
df_list = []

# Duyệt qua tất cả các file trong thư mục
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)
        df_list.append(df)

# Kết hợp tất cả các DataFrame thành một DataFrame lớn (nếu cần)
combined_df = pd.concat(df_list, ignore_index=True)
combined_df = combined_df.dropna()
combined_df = combined_df.iloc[1:]
# Hiện thị DataFrame
print(combined_df)

                                           joined_words  prediction
1     ơi đãi năm mới gói trọn_vẹn giảm chỉ từ mua tạ...         1.0
2                 runo là tiền_vệ ngu nhất mọi thời_đại         0.0
3                 thích mu nhưng ghét thằng ăn_hại runo         0.0
4                                       tôi phan mu năm         0.0
5                                           mu_mai dinh         0.0
...                                                 ...         ...
1924  là fan mu tôi cũng chẳng hiểu tại_sao mu_lại t...         0.0
1925                                   fan mu điểm_danh         0.0
1926  chúc_mừng mu có điểm đầy quý_giá và rất vất_vả...         0.0
1928                                                 up         0.0
1929                                           đầu_tiên         0.0

[1894 rows x 2 columns]


In [ ]:
combined_df.to_csv('drive/MyDrive/IE212_project/content/spark_streaming_prediction.csv', index=False)